In [2]:
import hail as hl
import numpy as np
from hail.linalg import BlockMatrix

In [3]:
hl.init(log='/tmp/hail.log')
bucket= 'gs://nbaya/risk_gradients'

Running on Apache Spark version 2.4.5
SparkUI available at http://nb1-m.c.ukbb-round2.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.43-d7da1fd14f3c
LOGGING: writing to /tmp/hail.log


In [26]:
def get_X(ref_panel, overwrite=False):
    r'''
    Returns N_ref x M dim matrix of column-standardized genotypes of LD ref panel
    '''
    X_bm_path = f'{bucket}/{ref_panel}.X.bm'
    
    if overwrite or not hl.hadoop_is_file(f'{X_bm_path}/_SUCCESS'):
        mt = hl.import_plink(bed=f'{bucket}/{ref_panel}.bed',
                             bim=f'{bucket}/{ref_panel}.bim',
                             fam=f'{bucket}/{ref_panel}.fam')
        
        mt = mt.annotate_rows(stats = hl.agg.stats(mt.GT.n_alt_alleles()))
        mt = mt.annotate_entries(X = (mt.GT.n_alt_alleles()-mt.stats.mean)/mt.stats.stdev)
        
        X = BlockMatrix.from_entry_expr(mt.X)
        X = X.T
        
        X.write(f'{bucket}/{ref_panel}.X.bm', overwrite=True)
        
    X = BlockMatrix.read(X_bm_path)
    
    return X

def get_beta(M, h2, model='inf', seed=None, block_size=None):
    r'''
    Returns M-dim vector of true SNP effect sizes
    '''
    if model=='inf': #infinitesimal model
        beta = np.sqrt(h2/M)*BlockMatrix.random(n_rows=M, 
                                                n_cols=1, 
                                                seed=seed,
                                                block_size=block_size,
                                                gaussian=True)
    return beta

def get_Z(N_r, block_size=None, seed=None):
    r'''
    Returns `N_r`-dim standard normal random vector
    '''
    Z = BlockMatrix.random(n_rows=N_r,
                           n_cols=1,
                           block_size=block_size, 
                           seed=seed,
                           gaussian=True) # N_r-dimensional standard normal random vector
    return Z

def get_rectangles(aldi_break_pts, M):
    r'''
    Get rectangles for `sparsify_rectangles`
    '''
    if not 0 in aldi_break_pts:
        aldi_break_pts.insert(0,0)
    if not M in aldi_break_pts:
        aldi_break_pts.append(M)
    rectangles = [[x1,x2]*2 for x1, x2 in zip(aldi_break_pts[:-1], aldi_break_pts[1:])]    
    rectangles.append([M]*4)
    return rectangles

def get_R(X, aldi_break_pts):
    r'''
    Returns M x M sparse LD matrix
    '''
    M = X.shape[1]
    rectangles = get_rectangles(aldi_break_pts=aldi_break_pts, M=M)  
    R = (X.T@X).sparsify_rectangles(rectangles)
    return R

def get_alpha(R, beta):
    r'''
    Returns M-dim vector of true marginal SNP effect sizes
    '''
    alpha = R@beta
    return alpha

def get_alphahat(alpha, N_d, N_r, X, Z):
    r'''
    Returns M-dim vector of estimated marginal SNP effect sizes
    '''
    alphahat = alpha + 1/np.sqrt(N_d*N_r)*X.T@Z
    return alphahat

def checkpoint_bm(bm, path, read_if_exists=True):
    if not hl.hadoop_is_file(f'{path}/_SUCCESS') or not read_if_exists:
        bm.write(path)
    bm = BlockMatrix.read(path)
    return bm

def get_yg(X, beta):
    r'''
    Returns genetic component of trait
    '''
    yg = X@beta
    return yg

def corrcoef(R, beta, betahat):
    r'''
    NOTE: Assumes that phenotype y = R@beta + E, where E is independent of R@alpha, has variance 1
    '''
    corr = (beta.T@R@betahat)/(betahat.T@R@betahat)**(1/2)
    return corr

In [55]:
ref_panel = '1kg_eur.chr22'

X = get_X(ref_panel=ref_panel) # column-standardized genotypes of LD reference panel
X_np = X.to_numpy()
X = BlockMatrix.from_numpy(X_np, block_size=2000)
block_size = X.block_size

N_r = X.shape[0] # number of samples in LD reference panel
M = X.shape[1] # number of variants

aldi_break_pts = [x*1000 for x in list(range(round(M/1000)))] # list of break points delimiting approx LD-independent regions, i-th entry represents SNP index of start of i-th LD block

R = get_R(X=X, aldi_break_pts=aldi_break_pts) # MxM LD matrix

R_path = f'{bucket}/{ref_panel}.R.blocksize_{block_size}.bm'
R = checkpoint_bm(bm=R, 
                  path=R_path, 
                  read_if_exists=True)

2020-06-29 20:56:19 Hail: INFO: wrote matrix with 46615 rows and 46615 columns as 24 blocks of size 2000 to gs://nbaya/risk_gradients/1kg_eur.chr22.R.blocksize_2000.bm


In [56]:
seed = 3
h2 = 0.5 # SNP heritability of trait
model = 'inf'

beta = get_beta(M=M, 
                h2=h2, 
                model=model, 
                block_size=block_size,
                seed=seed) # true SNP effect sizes


alpha = get_alpha(R, beta) # M-dimensional true marginal SNP effect sizes

N_d = 1000

Z = get_Z(N_r, block_size=block_size, seed=seed) # N_r-dimensional standard normal random vector
    
alphahat = get_alphahat(alpha=alpha,
                        N_d=N_d,
                        N_r=N_r,
                        X=X,
                        Z=Z)

alphahat_path = f'{bucket}/{ref_panel}.alphahat.h2_{h2}.{model}.Nd_{N_d}.blocksize_{block_size}.seed_{seed}.bm'
alphahat = checkpoint_bm(bm=alphahat, 
                         path=alphahat_path, 
                         read_if_exists=True)

2020-06-29 20:56:28 Hail: INFO: wrote matrix with 46615 rows and 1 column as 24 blocks of size 2000 to gs://nbaya/risk_gradients/1kg_eur.chr22.alphahat.h2_0.5.inf.Nd_1000.blocksize_2000.seed_3.bm


In [57]:
yg = get_yg(X=X, beta=beta)    
yg_np = np.squeeze(yg.to_numpy())

In [58]:
yhat = get_yg(X=X, beta=alphahat)
yhat_np = np.squeeze(yhat.to_numpy())

In [59]:
yg_yhat_corr = np.corrcoef(yg_np, yhat_np)[0,1]

In [60]:
print(yg_yhat_corr) #0.7305989661892002 when h2=0.5, N_d=1000, yhat beta=alphahat, seed=3, block_size=2000

0.7160533613011759


In [54]:
print(yg_yhat_corr) #0.7305989661892002 when h2=0.5, N_d=1000, yhat beta=alphahat, seed=3

0.7084164499826328


In [49]:
print(yg_yhat_corr) #0.7305989661892002 when h2=0.1, N_d=1000, yhat beta=alphahat, seed=1

0.7305989661892002


In [42]:
print(yg_yhat_corr) #0.7308640320155368 when h2=1, N_d=1000, yhat beta=alpha, seed=1

0.7308640320155368


In [39]:
print(yg_yhat_corr) #0.7038418502685398 when h2=1, N_d=1000, yhat beta=alphahat, seed=1

0.7307894787237522


In [19]:
print(yg_yhat_corr) #0.7038418502685398 when h2=, N_d=1000, yhat beta=alpha, seed=1

0.7046857428188459


In [181]:
print(yg_yhat_corr) #0.7038418502685398 when h2=0.5, yhat beta=alphahat

0.7038418502685398


In [178]:
print(yg_yhat_corr) #0.7038272731536311 when h2=0.5, yhat beta=alphahat

0.7038272731536311


In [173]:
print(yg_yhat_corr) #0.7038418502685402 when h2=1, yhat beta=alpha

0.7038418502685402


In [ ]:
yg_yhat_corr = ()

In [77]:
beta = get_beta(M=M, h2=0.5) # true SNP effect sizes

In [78]:
aldi_break_pts = [x*1000 for x in list(range(round(M/1000)))] # list of break points delimiting approx LD-independent regions, i-th entry represents SNP index of start of i-th LD block

R = get_R(X=X, aldi_break_pts=aldi_break_pts)

In [85]:
alpha = get_alpha(R=R, beta=beta) # true marginal SNP effect sizes

In [87]:
N_d = 1000

Z = get_Z(N_r=N_r)
alphahat = get_alphahat(alpha=alpha, 
                        N_d=N_d, 
                        N_r=N_r, 
                        X=X, 
                        Z=Z)

In [88]:
alphahat.write(f'{bucket}/tmp)

(46615, 1)

In [55]:
rectangles

[[0, 1000, 0, 1000],
 [1000, 2000, 1000, 2000],
 [2000, 3000, 2000, 3000],
 [3000, 4000, 3000, 4000],
 [4000, 5000, 4000, 5000],
 [5000, 6000, 5000, 6000],
 [6000, 7000, 6000, 7000],
 [7000, 8000, 7000, 8000],
 [8000, 9000, 8000, 9000],
 [9000, 10000, 9000, 10000],
 [10000, 11000, 10000, 11000],
 [11000, 12000, 11000, 12000],
 [12000, 13000, 12000, 13000],
 [13000, 14000, 13000, 14000],
 [14000, 15000, 14000, 15000],
 [15000, 16000, 15000, 16000],
 [16000, 17000, 16000, 17000],
 [17000, 18000, 17000, 18000],
 [18000, 19000, 18000, 19000],
 [19000, 20000, 19000, 20000],
 [20000, 21000, 20000, 21000],
 [21000, 22000, 21000, 22000],
 [22000, 23000, 22000, 23000],
 [23000, 24000, 23000, 24000],
 [24000, 25000, 24000, 25000],
 [25000, 26000, 25000, 26000],
 [26000, 27000, 26000, 27000],
 [27000, 28000, 27000, 28000],
 [28000, 29000, 28000, 29000],
 [29000, 30000, 29000, 30000],
 [30000, 31000, 30000, 31000],
 [31000, 32000, 31000, 32000],
 [32000, 33000, 32000, 33000],
 [33000, 34000, 33000

In [68]:
R.checkpoint(_read_if)

In [69]:
R.is_sparse

False

In [67]:
R.sparsify_rectangles(rectangles)

In [20]:
nd = np.array([[ 1.0,  2.0,  3.0,  4.0],
               [ 5.0,  6.0,  7.0,  8.0],
               [ 9.0, 10.0, 11.0, 12.0],
               [13.0, 14.0, 15.0, 16.0]])

In [22]:
bm = BlockMatrix.from_numpy(nd, block_size=2)

In [25]:
bm[1:2]

TypeError: __getitem__: parameter 'indices': expected tuple[(int or slice[(None or int), (None or int), (None or int)])], found slice: slice(1, 2, None)

In [23]:
bm.sparsify_rectangles([[0, 1, 0, 1], [0, 3, 0, 2], [1, 2, 0, 4]]).to_numpy()

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.],
       [ 9., 10.,  0.,  0.],
       [13., 14.,  0.,  0.]])

In [36]:
rectangles[-1]

[46000, 46616, 46000, 46616]

In [ ]:

bm = BlockMatrix.from_numpy(nd, block_size=2)

In [23]:
X.shape

(379, 46615)

In [10]:
# Dimensions: 379 samples, 3382774 variants
X = hl.linalg.BlockMatrix.read(f'{gs_bucket}/{ref_panel}.X.bm')
X = X.T

N = X.shape[0]
M = X.shape[1]
print(f'N samples:  {N}\nM variants: {M}')

N samples:  46615
M variants: 379


In [14]:
Z = np.random.multivariate_normal(mean=np.zeros(shape=N),
                                  cov=np.identity(n=N))
Z = hl.linalg.BlockMatrix.from_numpy(Z).T

In [15]:
E = 1/np.sqrt(N)*X.T@Z
E_bm_path = f'{gs_bucket}/{ref_panel}.E.bm'
if not hl.hadoop_is_file(f'{E_bm_path}/_SUCCESS'):
    E.write(E_bm_path)
E = BlockMatrix.read(E_bm_path)

In [8]:
E.shape

(3382774, 1)

In [54]:
seed = 1
np.random.seed(seed=seed)
h2 = 0.2
alpha = np.random.normal(loc=0, scale=np.sqrt(h2/M), size=M)
# pi = 0.01
# alpha = np.random.normal(loc=0, scale=np.sqrt(h2/(pi*M)), size=M)
alpha_np = np.random.normal(loc=0, scale=np.sqrt(h2/M), size=M)
alpha = BlockMatrix.from_numpy(alpha_np).T

In [66]:
# genetic component of phenotype
yg = (X@alpha)
yg_np = yg.to_numpy()

In [87]:
X_tb = X[:,0].to_table_row_major()

2020-04-30 20:28:59 Hail: INFO: wrote matrix with 379 rows and 1 column as 1 block of size 4096 to hdfs://nb1-m/tmp/hail.fds3ha22W58B/ullhDYc9VN


In [88]:
X_tb = X_tb.annotate(norm_gt = X_tb.entries[0])

In [90]:
X_tb.aggregate(hl.agg.stats(X_tb.norm_gt))

Struct(mean=-1.1248697663748289e-16, stdev=0.9999999999999993, min=-0.7635916923345639, max=2.046129194993589, n=379, sum=-4.263256414560601e-14)

In [62]:
beta = (1/N)*X.T@(yg)

# beta_bm_path = f'{gs_bucket}/{ref_panel}.beta.seed_{seed}.bm'
# if not hl.hadoop_is_file(f'{beta_bm_path}/_SUCCESS'):
#     beta.write(beta_bm_path, overwrite=True)
# beta = BlockMatrix.read(beta_bm_path)

In [64]:
beta_np = beta.to_numpy()

2020-04-30 20:02:10 Hail: INFO: BlockMatrix multiply: writing right input with 379 rows and 1 cols (1 blocks of size 4096) to temporary file hdfs://nb1-m/tmp/hail.fds3ha22W58B/pq8v2RhC0E.bm
2020-04-30 20:05:01 Hail: INFO: wrote matrix with 379 rows and 1 column as 1 block of size 4096 to hdfs://nb1-m/tmp/hail.fds3ha22W58B/pq8v2RhC0E.bm


In [65]:
beta_np

array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]])

In [20]:
tb = beta.to_table_row_major()

2020-04-30 19:13:49 Hail: INFO: Hail temporary directory: hdfs://nb1-m/tmp/hail.fds3ha22W58B
2020-04-30 19:14:04 Hail: INFO: BlockMatrix multiply: writing right input with 379 rows and 1 cols (1 blocks of size 4096) to temporary file hdfs://nb1-m/tmp/hail.fds3ha22W58B/tYlGDTOCqR.bm


KeyboardInterrupt: 

In [25]:
N_d = 10000
betahat = beta + (1/np.sqrt(N_d))*E

In [ ]:
alphahat = betahat

In [33]:
yhat = X@alphahat

In [34]:
yhat_np = yhat.to_numpy()

2020-04-30 19:25:53 Hail: INFO: BlockMatrix multiply: writing right input with 379 rows and 1 cols (1 blocks of size 4096) to temporary file hdfs://nb1-m/tmp/hail.fds3ha22W58B/j1e7TNtDf4.bm
2020-04-30 19:28:35 Hail: INFO: wrote matrix with 379 rows and 1 column as 1 block of size 4096 to hdfs://nb1-m/tmp/hail.fds3ha22W58B/j1e7TNtDf4.bm
2020-04-30 19:28:36 Hail: INFO: BlockMatrix multiply: writing right input with 3382774 rows and 1 cols (826 blocks of size 4096) to temporary file hdfs://nb1-m/tmp/hail.fds3ha22W58B/MLp1fmD5z5.bm
2020-04-30 19:28:52 Hail: INFO: wrote matrix with 3382774 rows and 1 column as 826 blocks of size 4096 to hdfs://nb1-m/tmp/hail.fds3ha22W58B/MLp1fmD5z5.bm


In [30]:
yg = X@alpha

In [53]:
alpha

In [32]:
yg_np = yg.to_numpy()

In [48]:
y_np = yg_np + np.random.normal(loc=0,scale=np.sqrt(1-h2), size=yg_np.shape)
np.corrcoef(y_np, yg_np)

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [28]:
r2_prs = ((alpha.T@X.T)())/(alphahat.T@X.T@X@alphahat)

TypeError: 'BlockMatrix' object is not callable

In [ ]:
tb = r_prs.to_table_row_major()

2020-04-27 13:21:05 Hail: INFO: Hail temporary directory: hdfs://nb3-m/tmp/hail.BHNrjsa2Wvzz
2020-04-27 13:21:17 Hail: INFO: BlockMatrix multiply: writing left input with 3382774 rows and 3382774 cols (682276 blocks of size 4096) to temporary file hdfs://nb3-m/tmp/hail.BHNrjsa2Wvzz/3vSmW1MWwt.bm


In [ ]:
tb.show()

In [ ]:
tb_betahat = betahat.to_table_row_major()

In [ ]:
tb_betahat.show()

In [17]:
r_prs = r_prs.checkpoint(f'{gs_bucket}/tmp.1kg_eur.r_prs.bm')

KeyboardInterrupt: 

In [12]:
r_prs = BlockMatrix.to_numpy(r_prs)

KeyboardInterrupt: 

In [ ]:
r_prs.shape

In [ ]:
r_prs